In [59]:
# [1]
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [60]:
# [2]
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [61]:
# [3]
len(words)

32033

In [62]:
# [4]
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [63]:
# [5]
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
    
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [64]:
# [6]
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [65]:
# [7]
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [66]:
# [8]
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [67]:
# [9]
C = torch.randn((27, 2))
C

tensor([[ 0.2963,  1.4077],
        [ 0.6609,  0.7253],
        [ 2.3105, -0.6289],
        [ 0.4021,  1.9937],
        [ 1.3615,  1.4883],
        [ 0.2343,  1.1642],
        [-0.9046,  0.5617],
        [-0.3937, -0.1113],
        [ 1.5597,  1.1442],
        [ 1.2305, -0.0508],
        [-0.0654,  2.4863],
        [-2.2601, -0.8691],
        [-0.7012,  0.4060],
        [-0.8170,  0.5588],
        [ 0.5569,  0.3866],
        [ 0.4946, -0.6509],
        [-0.4592,  0.0503],
        [ 0.2306, -0.6104],
        [-0.2054, -2.0566],
        [-0.6436,  0.4403],
        [ 0.4802,  0.0979],
        [ 0.0653,  0.2967],
        [ 0.3053,  0.4035],
        [ 1.0317, -1.3250],
        [-1.2052,  0.8432],
        [-0.8551,  0.4001],
        [-0.0287, -0.2313]])

In [68]:
# [10]
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [69]:
# [11]
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [70]:
# [12]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [71]:
h

tensor([[-0.9423,  0.4984,  0.9964,  ...,  0.7980,  0.9996, -0.2542],
        [-0.9670,  0.4415,  0.9954,  ...,  0.8222,  0.9995, -0.1471],
        [-0.9962, -0.3607,  0.6992,  ...,  0.9635,  0.9947, -0.1038],
        ...,
        [ 0.9971,  0.9967, -0.1735,  ..., -0.9826,  0.2553, -0.8618],
        [-0.9747,  0.3809,  0.9994,  ...,  0.0617,  0.9567,  0.9872],
        [-0.5380,  0.0786,  0.9969,  ..., -0.1847,  0.9978, -0.9681]])

In [72]:
h.shape

torch.Size([32, 100])

In [73]:
W2 = torch.randn((100, 100))
b2 = torch.randn(100)

In [74]:
logits = h @ W2 + b2

In [75]:
logits.shape

torch.Size([32, 100])

In [76]:
counts = logits.exp()

In [77]:
prob = counts / counts.sum(1, keepdims=True)

In [78]:
prob.shape

torch.Size([32, 100])

In [79]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.7445)

In [80]:
# --------------- now made respectable :) --------------------

In [81]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [82]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [83]:
sum(p.nelement() for p in  parameters) # number of parameters in total

3481

In [84]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)

Building makemore Part 2: MLP -> 32:00